In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

##  Работа с текстовыми данными 
Будем работать с данными с семинара, но использовать все категории текстов:

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True,random_state=7)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True,random_state=7)#

In [3]:
X_train=twenty_train.data
X_test=twenty_test.data
y_train=twenty_train.target
y_test=twenty_test.target

### 1. Мешок слов (1 балл)
Воспользуйтесь мешком слов  для признакового описания документа. Постройте логистическую регрессию с параметрами по умолчанию.  Какова доля верных предсказаний на тестовой выборке?

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [16]:
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_test_counts = count_vect.transform(X_test)

In [17]:
lr = LogisticRegression()
lr.fit(X_train_counts,y_train)
y_pred = lr.predict(X_test_counts)
print("Доля верных предсказаний = ",metrics.accuracy_score(y_test, y_pred))

Доля верных предсказаний =  0.8069569835369091


### 2. TF-IDF (1 балл)
Теперь в качестве признаков возьмите величины tf-idf. Как поменялось качество классификатора?

In [22]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
lr.fit(X_train_tfidf,y_train)
y_pred_tfidf = lr.predict(X_test_tfidf)
print("Доля верных предсказаний = ",metrics.accuracy_score(y_test, y_pred_tfidf))

Доля верных предсказаний =  0.8279341476367499


### 3.  Матрица ошибок (1 балл)
Выведите матрицу ошибок. Определите три пары категорий, которые модель путает чаще всего.  

In [71]:
matrix_of_errors = metrics.confusion_matrix(y_test,y_pred)
print( matrix_of_errors)

[[242   2   1   0   1   2   2   0   1   0   0   1   1   6   3  16   2   7
    1  31]
 [  2 294  13   7   7  18   5   7   0   7   0   5  12   0   3   3   0   2
    2   2]
 [  1  28 270  46  12  13   2   4   1   6   1   1   4   0   1   0   2   0
    1   1]
 [  1  11  26 271  26   7  14   4   1   0   0   3  27   0   0   0   0   0
    1   0]
 [  1   5   3  20 311   0   9   3   1   7   1   1  16   1   0   0   2   0
    3   1]
 [  1  44  38   8   4 277   4   1   3   1   1   1   5   2   3   1   0   0
    0   1]
 [  1   2   2   7   8   2 352   5   0   1   1   1   6   1   1   0   0   0
    0   0]
 [  0   3   0   3   2   0  11 340   6   4   0   0  17   4   1   1   2   0
    2   0]
 [  0   1   0   2   2   0   4   8 372   1   0   0   3   0   0   3   0   0
    1   1]
 [  2   1   1   0   3   0   5   3   0 356  16   0   1   2   1   2   0   2
    2   0]
 [  0   0   1   0   4   2   3   0   0   9 373   1   0   0   0   1   2   1
    2   0]
 [  0   5   2   3   5   2   4   2   1   3   1 350   7   1   0   1

In [74]:
matrix_of_errors = metrics.confusion_matrix(y_test,y_pred)
maximum = 0
for i in range(3):
    maximum = 0
    for i in range(len(matrix_of_errors)):
        for j in range(len(matrix_of_errors)):
            if i != j and (matrix_of_errors[i][j]+ matrix_of_errors[j][i] > maximum):
                maximum = matrix_of_errors[i][j]+ matrix_of_errors[j][i]
                i_max = i
                j_max = j
    matrix_of_errors[i_max][j_max] = 0
    matrix_of_errors[j_max][i_max] = 0
    print(twenty_train.target_names[i_max], "и" ,twenty_train.target_names[j_max], "перепутали ",maximum, "раза")

talk.politics.guns и talk.politics.misc перепутали  92 раза
comp.os.ms-windows.misc и comp.sys.ibm.pc.hardware перепутали  72 раза
alt.atheism и talk.religion.misc перепутали  63 раза


### 4.Определение наиболее важных признаков (слов) (2 балла)
Выберите категорию. Для гиперплоскости, отделяющей класс этой категории от остальных, определите 5 признаков, которые имеют наибольшие коэффициенты в линейной модели и 5 признаков, которые имеют наименьшие коэффициенты:
* определите слова, котором соответствуют наибольшие и наименьшие коэффициенты в линейной модели;
* напишите  функцию поиска ключа в словаре по заданному листу значений. 